In [0]:
import dlt
from pyspark.sql.functions import col, count, avg, round, sum, current_timestamp, expr

# GOLD DIMENSIONS (Analytical Schemas)

In [0]:
@dlt.table(name="dim_geography", comment="Gold Dimension for city coordinate mapping.")
def dim_geography():
    return dlt.read("silver_geography")

In [0]:
@dlt.table(name="dim_catalogs", comment="Gold Dimension for vehicle specifications and body types.")
def dim_catalogs():
    return dlt.read("silver_catalogs")

In [0]:
@dlt.table(name="dim_car_content", comment="Gold Dimension for car descriptions and photo counts.")
def dim_car_content():
    return dlt.read("silver_car_content")

# GOLD FACT TABLE (The central analytical hub)

In [0]:
@dlt.table(
    name="fact_car_sales",
    comment="Central Fact table joining transactions with all dimension keys.",
    table_properties={"pipelines.autoOptimize.zOrderCols": "transaction_id"}
)
def fact_car_sales():
    # Load Silver Sources
    sales = dlt.read("silver_transactions")
    geo = dlt.read("dim_geography")
    catalog = dlt.read("dim_catalogs")
    content = dlt.read("dim_car_content")
    
    return (
        sales
        # Join with Geography on geo_id
        .join(geo, "geo_id", "left")
        # Join with Catalog on composite key (marka, model)
        .join(catalog, ["marka", "model"], "left")
        # Join with Content on transaction_id/car_id
        .join(content, sales.transaction_id == content.car_id, "left")
        .select(
            sales["*"],
            geo.city_name,
            geo.lat,
            geo.lon,
            catalog.generation,
            catalog.body_type,
            content.description,
            content.photo_count,
            current_timestamp().alias("gold_load_timestamp")
        )
    )



# KPI AGGREGATION TABLES

In [0]:
# KPI 1: Geographical Market Performance
@dlt.table(name="gold_kpi_geo_performance", comment="Sales volume and market value by city.")
def gold_kpi_geo_performance():
    return (
        dlt.read("fact_car_sales")
        .groupBy("city_name")
        .agg(
            count("transaction_id").alias("listing_count"),
            round(avg("cost"), 2).alias("avg_listing_price"),
            sum("cost").alias("total_market_value")
        )
    )

In [0]:
# KPI 2: Brand & Body Type Popularity
@dlt.table(name="gold_kpi_brand_popularity", comment="Inventory distribution by brand and body type.")
def gold_kpi_brand_popularity():
    return (
        dlt.read("fact_car_sales")
        .groupBy("marka", "body_type")
        .agg(
            count("transaction_id").alias("total_listings"),
            round(avg("probeg"), 0).alias("avg_mileage")
        )
    )

In [0]:
# KPI 3: Pricing Strategy by Vehicle Age
@dlt.table(name="gold_kpi_price_trends", comment="Price trends categorized by car manufacturing year.")
def gold_kpi_price_trends():
    return (
        dlt.read("fact_car_sales")
        .groupBy("year")
        .agg(
            round(avg("cost"), 2).alias("avg_price"),
            round(avg("probeg"), 2).alias("avg_mileage")
        )
        .sort("year")
    )

In [0]:
# KPI 4: Visual Content Impact (Liquid Clustering enabled)
@dlt.table(
    name="gold_kpi_content_impact",
    comment="Analyzes if higher photo counts correlate with higher listing prices.",
    cluster_by=["marka"]
)
def gold_kpi_content_impact():
    return (
        dlt.read("fact_car_sales")
        .withColumn("photo_range", 
            expr("CASE WHEN photo_count = 0 THEN 'No Photos' WHEN photo_count < 5 THEN 'Low' ELSE 'High' END"))
        .groupBy("marka", "photo_range")
        .agg(
            count("transaction_id").alias("listing_count"),
            round(avg("cost"), 2).alias("avg_price")
        )
    )